# Title: BIDS Containerization- Running hardcoded scripts

Author: Nichollette Acosta <br>
Description: This is a run down of the workflow used to convert dicom files into a BIDS format. This goes through each individual script required to complete a dicom to BIDS conversion on a high performance computing cluster (HPC). 







## Get DICOMS from OsiriX 

We are going to start with a quick workflow of grabbing DICOM data from the OsiriX web API. Here we assume that you have your dicom data on OsiriX and have access to a web API. 

### Title: osirix_scrape.sh 
Description: Used to connect to the OsiriX web API, take a <i>given</i> target ID and find all matching subjects in OsiriX and copy them to your HPC. <br>
Required Input: <br>
-Osirix username<br>
-Osirix password<br>
-Target subject keyword<br> 
-HPC output directory path<br>
-HPC password<br>


In [2]:
###########################################################################################################################
"""
# @author: Nichollette Acosta
# @title: OsiriX Subject Scraper
  @Description: This script is used to scrape OsiriX viewer and grab
                our subject ids. We will give it a specifc ID to gather,
                then we will iterate through our IDS and run our setup_subjects.py
                on the given ID.

"""

###########################################################################################################################



import dryscrape
from bs4 import BeautifulSoup
###########################################################################################################################


### Here is where we add our base and target OsiriX URLs - we are using OsiriX web

## This address base is unique for your viewer
url_base = 'http://152.2.112.193:3333/main'
## The target base - this is the URL where all your subjects are listed
url_target = "http://152.2.112.193:3333/studyList?browse=all"



###########################################################################################################################
# Here is where we are opening up our Osirix Viewer so we can grab data
dryscrape.start_xvfb()
session = dryscrape.Session()
session.visit(url_base)



###########################################################################################################################

# Here is where you enter your OsiriX username -- add here your information

x = session.at_xpath('//*[@name="username"]')
x = session.at_xpath('//*[@name="password"]')

# ENTER YOUR USERNAME HERE 
username = 'nibl'
x.set(username)


# ENTER YOUR PASSWORD HERE
password = 'eatthisnotthat'
x.set(password)

###########################################################################################################################
login = session.at_xpath('//*[@name="login"]')
login.click()

session.visit(url_target)
page = session.body()
soup = BeautifulSoup(page, 'lxml')



###########################################################################################################################

# --Need to  edit this for reproducibility

# declaring a list that will hold our subject subject ids
target_ids = []
# using the BeautifulSoup package to find ALL of our ids in OsiriX
patient_tag_ids = soup.find_all('span', class_="study-list-patient-id")


# ENTER YOUR TARGET ID KEYWORD BELOW
target_id = "SM"


# here we loop through our ids we retrieved above, and we find our target ids and append
# them to our target_id list
for id in patient_tag_ids:
    if target_id in id.text:
        target_ids.append(id.text)
        
###########################################################################################################################

# Here is where I am adding ids to a file -->
# --> NEED TO : REPLACE WITH GET DICOMS LINE BELOW:
# python setup_subjects.py --getdata --keepdata --osirix_username nibl --osirix_password eatthisnotthat --osirix_subjName "BF20v2 18Dec2015" --studyname SugarMama -s "BF20v2" -o
# Then add command below to add data to HPC
# scp -r -p SM* nbytes@ht0.renci.org:/projects/niblab/test/output/SugarMama2
#    sshpass -p 'sweetbbcs'
#    sudo rm -rf SM*


"""
file = open("SM_names.txt", "w")

for id in SMpatient_ids:
    print(id)
    id_out = id.split(' ')[0]
    file.write(id + "," + id_out + "\n")

file.close()
"""


ModuleNotFoundError: No module named 'dryscrape'

## BIDS Conversion

Description: The scripts below are used to run a BIDS conversion on data in the HPC. Here we assume you are logged on to your HPC and run the 

### Title: converterfile.py 
Original can be found here: http://nipy.org/workshops/2017-03-boston/lectures/bids-heudiconv/#19 <br>
BIDS Specifications Here: http://bids.neuroimaging.io/bids_spec1.0.2.pdf <br>
Description: This file has to be customized the most. This script is used for for the Heudiconv converter that we containerize with singularity. The script requires specific parameters for your data. This is where the folders are created for BIDS. Please read the references above for clarity.


In [ ]:
#!/usr/bin/env bash

######################################################################################################################################
"""
# @Author: Nichollette Acosta
# @Title: Heuristic Conversion File
# @Description:  This script is unique for your subject. The parameters below need to
                be made for yoour specific data. This is used when we run the bidsconversion file. 
"""
######################################################################################################################################



import os
subject = os.environ["value"]

def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes


def infotodict(seqinfo):
    # create directories

    # anat/
    t1 = create_key('anat/sub-' + subject+ '_T1w')


    # fmap/
    fmap_phase = create_key('fmap/sub-' + subject + '_phasediff')
    fmap_magnitude = create_key('fmap/sub-' + subject + '_magnitude')


    # func/
    rest = create_key('func/sub-' + subject + '_task-rest_bold')
    train = create_key('func/sub-' + subject + '_task-train_bold')
    predict = create_key('func/sub-' + subject + '_task-predictionerror_bold')




    info = {t1: [],  fmap_phase: [], fmap_magnitude: [], rest: [], train: [], predict: [] }

    for s in seqinfo:
        print(s)
        if (s.dim3 == 192) and ('t1' in s.protocol_name):
            info[t1].append(s.series_id)  ## append if multiple series meet criteria
        if (s.dim3 == 72) and ('field_mapping' in s.protocol_name):
            info[fmap_magnitude].append(s.series_id)  ## append if multiple series meet criteria
        if (s.dim3 == 36) and ('field_mapping' in s.protocol_name):
            info[fmap_phase].append(s.series_id)  # append if multiple series meet criteria
        if (s.dim4 == 348) and ('bold' in s.protocol_name):
            info[train].append(s.series_id)  # append if multiple series meet criteria
        if (s.dim4 == 147) and ('resting' in s.protocol_name):
            info[rest].append(s.series_id)  # append if multiple series meet criteria
        if (s.dim4 == 193) and ('bold' in s.protocol_name):
            info[predict].append(s.series_id)  # append if multiple series meet criteria
        ##if (s.dim4 == 170) and ('bold' in s.protocol_name):
            #info[pictures].append(s.series_id)  # append if multiple series meet criteria

    return info


### Title: bids_conversion.py 


In [4]:
######################################################################################################################################
"""
# @Author: Nichollette Acosta
# @Title: Bids Converter
# @Description:  This will shell into our singularity image and run the heudiconv converter
#                on our data
"""
######################################################################################################################################

# RUN SINGULARITY --
# Right now I have a singularity shell ..... look into making a run file for This

singularity shell -B /projects/niblab/test:/test heudiconv.simg



cd /lab_data/SugarMama
files=(SM*)
cd ../../
num=0
for f in ${files[@]};do
num=$(($num+1))
value=$(printf "%02d" $num)
export value
heudiconv -b -d /lab_data/eric_data/dicoms/wave1/dicoms/007/*.dcm -s $f -f /test/heuristic_converter00.py -c dcm2niix -b  -o /test/output/SugarMama2/sub-${value}
echo $value
done
heudiconv -b -ss 1 --overwrite -d /lab_data/{subject}/dicoms/wave1/dicoms/007/*.dcm -s eric_data -f /test/NIBL/ericdata_converter.py -c dcm2niix -b  -o /test/output/eric_test/sub-07


SyntaxError: invalid syntax (<ipython-input-4-d16850d25fbe>, line 13)

### Title: bids_polish.py


In [5]:
#!/usr/bin/env bash

######################################################################################################################################
"""
# @Author: Nichollette Acosta
# @Title: Bids Polish
# @Description:  This script will grab the participants and make a participants.tsv
                  file for our BIDS folder, as well as polish our BIDS conversion
"""

######################################################################################################################################


(echo -e "participant_id \t age \t sex") | column -t >> participants.txt

# get subject / folder names
folders=(sub*)

# start loop
for f in ${folders[@]}; do
    cd $f

    #get participants
    subject=$(awk 'NR == 2 {print  $1}' participants.tsv)
    info=$(awk 'NR == 2 {print  " \t" $2 " \t" $3}' participants.tsv)
    info="${f}${info}"
    participant=$info
    #participants=(${participants[@]} $info)
    #echo ${particpants[@]}


    cd ..

    echo "$subject  $f" >> subject_conversion.txt

    echo  "$participant" >> participants.txt

    cd $f

    lsfor
    task="${f}task"

    rm task* README CHANGES data* part* $task*

    ls
    cd ..
done

column -t participants.txt >> participants.tsv
rm participants.txt


SyntaxError: invalid syntax (<ipython-input-5-f2f9f2d8c40a>, line 14)